In [1]:
!pip install py7zr

     |████████████████████████████████| 65 kB 2.2 MB/s 
     |████████████████████████████████| 1.9 MB 13.0 MB/s 
     |████████████████████████████████| 124 kB 55.9 MB/s 
     |████████████████████████████████| 2.2 MB 32.4 MB/s 
     |████████████████████████████████| 357 kB 55.2 MB/s 


In [2]:
import torch
from torch import nn
from torchvision import models,transforms,datasets
from torch import optim
import matplotlib.pyplot as plt
import py7zr
from collections import OrderedDict
import PIL
from torch.utils.data import Dataset
import os
import glob

In [3]:
zip_path='/content/drive/MyDrive/Applied DS-AI/Session7/Assignment/flower_data.7z'

In [4]:
# unzip file
with py7zr.SevenZipFile(zip_path,mode='r') as z:
  print('progressing unzip ...')
  z.extractall()
  print('done')

progressing unzip ...
done


In [5]:
data_dir='/content/flower_data/'

In [6]:
train_transofrm=transforms.Compose([transforms.RandomRotation(30),
                                   transforms.RandomResizedCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])
test_transofrm=transforms.Compose([transforms.Resize(255),
                                   transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])


In [7]:
import re
class get_data(Dataset):
  def __init__(self,path,transform):
    self.path=path
    self.transform=transform
    self.items=glob.glob(path+'/*/*')
    self.labels=self.get_all_labels()
  def get_all_labels(self):
    labels=set([])
    for i in self.items:
      temp_label=re.search('/\d*/',i)
      label=int(temp_label.group().replace('/',''))-1
      labels.add(label)
    return labels
  def __len__(self):
    return len(self.items)
  def __getitem__(self,ind):
    item=self.items[ind]
    temp_label=re.search('/\d*/',item)
    label=int(temp_label.group().replace('/',''))-1
    image=PIL.Image.open(item)
    image_tensor=self.transform(image)
    output=(image_tensor,label)
    return output

In [8]:
train_data=get_data(data_dir+'train',transform=train_transofrm)
test_data=get_data(data_dir+'test',transform=test_transofrm)
val_data=get_data(data_dir+'valid',transform=test_transofrm)

In [9]:
# train_data=datasets.ImageFolder(data_dir+'train',transform=train_transofrm)
# test_data=datasets.ImageFolder(data_dir+'test',transform=test_transofrm)
# val_data=datasets.ImageFolder(data_dir+'valid',transform=test_transofrm)
#problem happend here that not take labels from folders it makes labels like 0 in folders name 11  that could make confusing

In [10]:
train_loader=torch.utils.data.DataLoader(train_data,shuffle=True,batch_size=64)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=64)
val_loader=torch.utils.data.DataLoader(val_data,batch_size=64)

In [11]:
print(train_loader.dataset.labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101}


In [12]:
model=models.resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [13]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
for param in model.parameters():
    param.requires_grad=False
fc=nn.Sequential(OrderedDict([
                            ('fc1',nn.Linear(2048,1000)),
                            ('relu',nn.ReLU()),
                             ('fc2',nn.Linear(1000,500)),
                             ('relu1',nn.ReLU()),
                             ('fc3',nn.Linear(500,102)),
                             ('output',nn.LogSoftmax(dim=1))]))
model.fc=fc

In [15]:
import time
from tqdm import tqdm

In [16]:
# for device in ['cuda','cpu']:
#     criterion=nn.NLLLoss()
#     optimizer=optim.Adam(model.parameters(),lr=0.001)
#     model.to(device)
#     start=time.time()
#     for inputs,labels in train_loader:
#         inputs,labels=inputs.to(device),labels.to(device)
#         outputs=model(inputs)
#         loss=criterion(outputs,labels)
#         loss.backward()
#         optimizer.step()
#         break
#     end=time.time()
#     print(f"model took at {device} device {end-start} second at one batch")

In [18]:
#traing
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion=nn.NLLLoss().to(device)
optimizer=optim.Adam(model.parameters(),lr=0.001)
model.to(device)
print(device)
epochs=25
show_every=50

cuda


In [19]:
def get_labels(outputs,device='cpu'):
  label=[torch.argmax(t) for t in outputs]
  label=torch.tensor(label).to(device)
  return label

In [20]:
epoch_loss=0
step=0
total_train=len(train_loader.dataset)
total_test=len(test_loader.dataset)
total_val=len(val_loader.dataset)
for epoch in range(epochs):
    epoch_loss=0
    correct_t=0
    correct_sv_t=0
    correct_v=0
    total_val_loss=0
    for train_i,train_l in tqdm(train_loader):
        train_i,train_l=train_i.to(device),train_l.to(device)
        optimizer.zero_grad()
        t_outputs=model(train_i)  
        loss=criterion(t_outputs,train_l)
        loss.backward()
        optimizer.step()
        # epoch_loss+=loss.item()
        # q_t=[torch.argmax(t) for t in t_outputs]
        t_outputs_label=get_labels(t_outputs,device)
        correct_t+=(t_outputs_label==train_l).float().sum()
        correct_sv_t+=(t_outputs_label==train_l).float().sum()
        step+=1
        if step%show_every==0:
          print(f'\n show every : {show_every} tarin loss : {loss}, train accuracy: {(correct_sv_t/total_train)*100} %')
          correct_sv_t=0
    print(f'\n epoch : {epoch} tarin loss : {loss}, train accuracy: {(correct_t/total_train)*100} %')
    model.eval()
    print('validation')
    for val_i ,val_l in tqdm(val_loader):
        val_i,val_l=val_i.to(device),val_l.to(device)
        val_outputs=model(val_i)
        loss_val=criterion(val_outputs,val_l)
        total_val_loss+=loss_val.item()
        val_outputs_label=get_labels(val_outputs,device)
        correct_v+=(val_outputs_label==val_l).float().sum()
    print(f'\n epoch : {epoch} val loss : {total_val_loss}, val accuracy: {(correct_v/total_val)*100} %')
    model.train()

  0%|          | 0/103 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
 49%|████▊     | 50/103 [01:14<01:20,  1.51s/it]


 show every : 50 tarin loss : 3.1562530994415283, train accuracy: 6.135531425476074 %


 97%|█████████▋| 100/103 [02:28<00:04,  1.48s/it]


 show every : 50 tarin loss : 1.762081503868103, train accuracy: 20.863859176635742 %


100%|██████████| 103/103 [02:32<00:00,  1.48s/it]


...


 show every : 50 tarin loss : 0.20519082248210907, train accuracy: 44.06288146972656 %


100%|██████████| 103/103 [02:34<00:00,  1.50s/it]



 epoch : 24 tarin loss : 0.2795579135417938, train accuracy: 90.26251983642578 %
validation


100%|██████████| 13/13 [00:18<00:00,  1.46s/it]


 epoch : 24 val loss : 3.9988232478499413, val accuracy: 93.64303588867188 %


In [21]:
print('test')
total_test=len(test_loader.dataset)
total_test_loss=0
correct_test=0
model.eval()
for test_i ,test_l in tqdm(test_loader):
    test_i,test_l=test_i.to(device),test_l.to(device)
    test_outputs=model(test_i)
    loss_test=criterion(test_outputs,test_l)
    total_test_loss+=loss_test.item()
    test_outputs_label=get_labels(test_outputs,device)
    correct_test+=(test_outputs_label==test_l).float().sum()
print(f'\n  test loss : {total_test_loss}, test accuracy: {(correct_test/total_test)*100} %')

test


100%|██████████| 13/13 [00:19<00:00,  1.47s/it]


  test loss : 4.1911224983632565, test accuracy: 91.57508850097656 %


In [ ]:
#test accuarcy 91.5% which is very good 